# Update DOIs

In [6]:
import urllib
import urllib.request
import urllib.parse
import urllib.error

import numpy as np
import pandas as pd
import os


import requests
from bs4 import BeautifulSoup
import csv

from pathlib import Path

from start import data_path
import gather_documents
import clean_documents


In [7]:
data_path

'/Users/kla21002/Dropbox/Active/dofis/data/plans/'

In [8]:
# import cleaned exemptions list
docs_df = pd.read_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/doi_exemptions_and_dates.csv")
print(len(docs_df))
docs_df[docs_df.title == "South San Antonio ISD"]

938


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,title,Unnamed: 0_x,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,...,possible_laws,Unnamed: 0_y,term_year,term_month,finalize_year,finalize_month,term_phrase,finalize_phrase,date_year,date_month
681,681,682,683.0,693.0,693.0,South San Antonio ISD,130.0,614.0,614.0,614.0,...,"[25.0811, 21.003, 25.112, 25.1113, 21.051, 25....",130.0,2017.0,NaN,2017.0,April,NaN,NaN,NaN,NaN


In [6]:
# old districts with missing laws
missing_laws = docs_df[docs_df.possible_laws == '[]']
len(missing_laws)
missing_laws_list = list(missing_laws.title)
missing_laws_list

['Baird ISD']

## Check TEA website for new districts of innovation

In [7]:
url = "https://tea.texas.gov/Texas_Schools/District_Initiatives/Districts_of_Innovation/"
webcontent = urllib.request.urlopen(url).read()
soup = BeautifulSoup(webcontent, 'html.parser')
links = soup.find_all('a')
districts_list = [i for i in map(lambda x: x.get('title'), links) 
                  if i is not None and 'ISD' in i]
print("Number of districts on DOI website", len(districts_list))

Number of districts on DOI website 859


In [9]:
new_districts_list = []
for dist in districts_list:
    if dist not in list(docs_df.title):
        new_districts_list.append(dist)



In [15]:
print("Number of new districts: ", len(new_districts_list))
new_districts_list
new_districts_df = pd.DataFrame(new_districts_list)
new_districts_df.to_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/new_DOIS_2022-06-10.csv")

Number of new districts:  40


In [11]:
retired_districts_list = []
for dist in list(docs_df.title):
    if dist not in districts_list:
        retired_districts_list.append(dist)

In [16]:
print("Number of retired DOIs: ", len(retired_districts_list))
retired_districts_list
retired_districts_df = pd.DataFrame(retired_districts_list)
retired_districts_df.to_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/retired_DOIS_2022-06-10.csv")

Number of retired DOIs:  81


In [18]:
new_df = pd.DataFrame(new_districts_list, columns=['title'])
new_df.head()


,title
0,Austwell-Tivoli ISD
1,Balmorhea ISD
2,Bandera ISD
3,Borger ISD
4,Brady ISD


# Gather

In [19]:
first_level_links = gather_documents.FirstLevelLinks(url, print_interim=False)
first_level_df = first_level_links.docs_df.reset_index().rename(columns={'index': 'title'})
print(len(first_level_df))
first_level_df

/Users/kla21002/dofis/dofis/data_from_plans/gather_documents.py:109: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 109 of the file /Users/kla21002/dofis/dofis/data_from_plans/gather_documents.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


8


,title,link,type
0,Post ISD,https://1.cdn.edl.io/I2a9qAWX4QHUGdG5HRdQP38Ja...,docx
1,Clifton ISD,https://core-docs.s3.amazonaws.com/documents/a...,pdf
2,Moulton ISD,https://1.cdn.edl.io/muYNPRRDs7A0eAKh3p9RjqTpd...,pdf
3,City View ISD,https://drive.google.com/drive/folders/0B3zKmp...,google
4,Milano ISD,https://docs.google.com/document/d/1irpKI3kN9e...,google
5,Miller Grove ISD,https://docs.google.com/document/d/1VXGGMvUftk...,google
6,Whitehouse ISD,https://docs.google.com/document/d/1lxtD2uHrmK...,google
7,Winona ISD,https://docs.google.com/document/d/1kDyG_vaZ0F...,google


In [21]:
new_df = new_df.merge(first_level_df, how = 'left')
print(len(new_df))
new_df
new_df.to_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/update_links_2022-06-10.csv")

40


In [24]:
updated_df = pd.read_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/update_links_corrected_2022-06-10.csv")
updated_df = updated_df[['title', 'link']]
updated_df

,title,link
0,Austwell-Tivoli ISD,https://core-docs.s3.amazonaws.com/documents/a...
1,Balmorhea ISD,https://files.gabbart.com/160/balmorhea_approv...
2,Bandera ISD,https://4.files.edl.io/605a/11/01/21/161222-3f...
3,Borger ISD,https://cdn5-ss11.sharpschool.com/UserFiles/Se...
4,Brady ISD,https://cdn5-ss9.sharpschool.com/UserFiles/Ser...
5,Charlotte ISD,https://www.charlotteisd.net/pages/uploaded_fi...
6,Chilton ISD,https://core-docs.s3.amazonaws.com/documents/a...
7,Coldspring-Oakhurst CISD,https://docs.google.com/viewerng/viewer?url=ht...
8,Corpus Christi ISD,https://drive.google.com/file/d/1aMqKKzxEN0WZm...
9,Cotton Center ISD,https://drive.google.com/file/d/1AdTzXvoiGgdQ9...


## Extract

### Clean Text

In [25]:
texts = []
for link in updated_df.link:
    if not isinstance(link, str):
        texts.append("no link")
    elif isinstance(link, str):
        text = clean_documents.get_plain_text(link)
        texts.append(text)
updated_df['text'] = texts
updated_df = updated_df.fillna('No text') # replace None with 'None'
updated_df

,title,link,text
0,Austwell-Tivoli ISD,https://core-docs.s3.amazonaws.com/documents/a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Balmorhea ISD,https://files.gabbart.com/160/balmorhea_approv...,No text
2,Bandera ISD,https://4.files.edl.io/605a/11/01/21/161222-3f...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,Borger ISD,https://cdn5-ss11.sharpschool.com/UserFiles/Se...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,Brady ISD,https://cdn5-ss9.sharpschool.com/UserFiles/Ser...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5,Charlotte ISD,https://www.charlotteisd.net/pages/uploaded_fi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,Chilton ISD,https://core-docs.s3.amazonaws.com/documents/a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
7,Coldspring-Oakhurst CISD,https://docs.google.com/viewerng/viewer?url=ht...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
8,Corpus Christi ISD,https://drive.google.com/file/d/1aMqKKzxEN0WZm...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
9,Cotton Center ISD,https://drive.google.com/file/d/1AdTzXvoiGgdQ9...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [26]:
updated_df = clean_documents.remove_whitespace(updated_df, text_col = 'text')
updated_df.head()

,title,link,text
0,Austwell-Tivoli ISD,https://core-docs.s3.amazonaws.com/documents/a...,Austwell-Tivoli Independent School District Pr...
1,Balmorhea ISD,https://files.gabbart.com/160/balmorhea_approv...,No text
2,Bandera ISD,https://4.files.edl.io/605a/11/01/21/161222-3f...,BISD District of Innovation Plan--February 201...
3,Borger ISD,https://cdn5-ss11.sharpschool.com/UserFiles/Se...,Borger Independent School District District of...
4,Brady ISD,https://cdn5-ss9.sharpschool.com/UserFiles/Ser...,BRADY INDEPENDENT SCHOOL DISTRICT 1003 W. 11th...


## Extract laws and dates manually

In [28]:
updated_df['possible_laws'] = ""
updated_df['term_year'] = ""
updated_df['term_month'] = ""
updated_df['finalize_year'] = ""
updated_df['finalize_month'] = ""
updated_df['date_year'] = ""
updated_df['date_month'] = ""
updated_df.to_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/update_dates_and_laws_2022-06-10.csv")

Open update_dates_and_laws and save as update_dates_and_laws_corrected.csv. Correct term month and year, finalize month and year, and dates. If data is not available in plan, leave blank (for dates) or as an empty list (for laws)

# import and append

In [4]:
new_df = pd.read_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/update_dates_and_laws_corrected_2022-06-10.csv")
old_df = docs_df.loc[~docs_df.title.isin(new_districts_list)]
print(len(new_df))
print(len(old_df))

NameError: name 'docs_df' is not defined

In [30]:
final_df = old_df.append(new_df)
final_df.sample(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,title,Unnamed: 0_x,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,...,possible_laws,Unnamed: 0_y,term_year,term_month,finalize_year,finalize_month,term_phrase,finalize_phrase,date_year,date_month
743,744,745.0,755.0,755.0,Utopia ISD,24.0,3153.0,3153.0,3153.0,3505.0,...,"[25.0812, 21.003, 25.0811, 25.036]",68.0,2018.0,NaN,2018.0,November,NaN,NaN,NaN,NaN
287,288,289.0,293.0,293.0,Goliad ISD,530.0,543.0,543.0,543.0,557.0,...,"[37.152, 37.007, 25.0811, 21.003, 21.401, 21.0...",530.0,2017.0,NaN,2016.0,October,NaN,NaN,NaN,NaN
64,64,65.0,65.0,65.0,Bluff Dale ISD,758.0,3052.0,3052.0,3052.0,3391.0,...,"[25.113, 25.0811, 21.102, 21.002, 21.003, 21.0...",758.0,2019.0,February,2019.0,February,NaN,NaN,NaN,NaN
144,144,145.0,147.0,147.0,Clarendon CISD,676.0,3272.0,3272.0,3272.0,3640.0,...,"[25.0811, 25.082, 21.4021, 21.003, 21.401, 25....",676.0,2017.0,March,2017.0,March,NaN,NaN,NaN,NaN
207,208,209.0,213.0,213.0,Dodd City ISD,610.0,28.0,28.0,28.0,28.0,...,"[25.113, 21.003, 25.0811, 25.112]",610.0,2017.0,NaN,2017.0,January,NaN,NaN,NaN,NaN


In [31]:
len(docs_df)

898

In [32]:
docs_df[docs_df.title.str.contains(pat="Big Sandy", regex=False)]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,title,Unnamed: 0_x,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,...,text,p_innovation,possible_laws,Unnamed: 0_y,term_year,term_month,finalize_year,finalize_month,term_phrase,finalize_phrase
818,819,820,6,6,Big Sandy ISD (230901),NaN,NaN,NaN,NaN,NaN,...,No text,NaN,"[25.0811, 25.0812, 21.003, 21.057, 25.112, 25....",NaN,2017.0,NaN,NaN,NaN,NaN,NaN
819,820,821,7,7,Big Sandy ISD (187901),NaN,NaN,NaN,NaN,NaN,...,- 1 - DISTRICT OF INNOVATION PLAN BIG SANDY IS...,NaN,"[21.102, 21.003, 21.401, 21.055, 25.036, 21.04...",NaN,2020.0,NaN,NaN,NaN,"a “District of Innovation”, as authorized by H...",NaN


In [33]:
final_df = final_df[final_df.title != 'Big Sandy ISD'] 
final_df = final_df[final_df.title != 'Dawson ISD'] 

In [34]:
len(final_df)

938

In [36]:
final_df.to_csv("/Users/kla21002/Dropbox/Active/dofis/data/plans/doi_exemptions_and_dates.csv", encoding="utf-8")